In [11]:
import ctypes
import numpy as np
import imageio
import os

In [12]:
# Load the SDK
pco = ctypes.windll.LoadLibrary("PCO_CAM_SDK.dll")

# Define data types
USHORT = ctypes.c_ushort
DWORD = ctypes.c_uint32
HANDLE = ctypes.c_void_p
PVOID = ctypes.c_void_p

FileNotFoundError: Could not find module 'PCO_CAM_SDK.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
# Open the camera
cam = HANDLE()
error = pco.PCO_OpenCamera(ctypes.byref(cam), 0)
print("OpenCamera error code:", error)

# Arm the camera
error = pco.PCO_ArmCamera(cam)
print("ArmCamera error code:", error)

In [ ]:
# Allocate buffer
sBufNr = USHORT()
wXRes, wYRes = 1920, 1080  # use your actual resolution
image_size = wXRes * wYRes * 2  # 16-bit pixels

# Create empty buffer
buf = (ctypes.c_ushort * (wXRes * wYRes))()
buf_ptr = ctypes.cast(buf, PVOID)

error = pco.PCO_AllocateBuffer(cam, ctypes.byref(sBufNr), DWORD(image_size), ctypes.byref(buf_ptr), 0)
print("AllocateBuffer error:", error)

In [ ]:
# Set recording state to ON
error = pco.PCO_SetRecordingState(cam, 1)
print("SetRecordingState (start) error:", error)

# Trigger an image (depends on mode)
# If using software trigger:
pco.PCO_ForceTrigger(cam)

# Wait for image to be available (simplified)
import time
time.sleep(1)  # wait for image exposure

# Grab image
error = pco.PCO_GetImageEx(cam, sBufNr, 0, 1, 16)
print("GetImageEx error:", error)

# Save image
image_array = np.ctypeslib.as_array(buf).reshape((wYRes, wXRes))
imageio.imwrite("pco_image.tiff", image_array)
print("Image saved as pco_image.tiff")


In [ ]:
# Stop recording
error = pco.PCO_SetRecordingState(cam, 0)
print("SetRecordingState (stop) error:", error)

# Free buffer
pco.PCO_FreeBuffer(cam, sBufNr)

# Close camera
pco.PCO_CloseCamera(cam)